In [1]:
import torch 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
chars = ""
with open('data\Two sailor lads.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars, len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '3', '5', '7', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '§', 'Ñ', 'æ', 'é', 'ê', 'ñ', 'ö', '†', '‡', '\ufeff'] 86


In [3]:
##vocab
text_to_int = {char:idx for idx, char in enumerate(chars)}
int_to_text = {idx:char for idx, char in enumerate(chars)}

In [4]:
encode = lambda z: [text_to_int[char] for char in z]
decode = lambda z: ''.join(int_to_text[int] for int in z)

In [5]:
encoded = encode('hello')
encoded

[56, 53, 60, 60, 63]

In [6]:
decode(encoded)

'hello'

In [7]:
data = torch.tensor(encode(text), dtype = torch.long) #long seq of integers

In [8]:
print(data[:100])

tensor([85,  1,  1, 48, 39, 71, 63,  1, 38, 49, 57, 60, 63, 66,  1, 31, 49, 52,
        67, 11, 48,  0,  0,  1,  1, 20,  1, 38, 39, 34, 37, 44,  1, 34, 25,  0,
         0,  1,  1, 38, 39, 28, 37, 37, 28, 33, 26,  1, 20, 23, 41, 24, 33, 39,
        40, 37, 24, 38,  1, 34, 33,  1, 38, 24, 20,  1, 20, 33, 23,  1, 31, 20,
        33, 23,  0,  0,  0,  1,  1, 21, 44,  0,  0,  1,  1, 26, 34, 37, 23, 34,
        33,  1, 38, 39, 20, 21, 31, 24, 38,  9])


In [9]:
#validation and training split
split_index = int(0.8 * len(text))
train = data[:split_index]
test = data[split_index:]

In [10]:
print(len(train), len(test))

380708 95178


In [11]:
block_size = 8

x = train[:block_size]
y = train[1: block_size+1]

for i in range(block_size):
    print(f"input : {x[:i+1]} target: {y[i]}")

input : tensor([85]) target: 1
input : tensor([85,  1]) target: 1
input : tensor([85,  1,  1]) target: 48
input : tensor([85,  1,  1, 48]) target: 39
input : tensor([85,  1,  1, 48, 39]) target: 71
input : tensor([85,  1,  1, 48, 39, 71]) target: 63
input : tensor([85,  1,  1, 48, 39, 71, 63]) target: 1
input : tensor([85,  1,  1, 48, 39, 71, 63,  1]) target: 38


In [12]:
ix = torch.randint((len(data) - 8), (4,))  #indices
ix
train[ix[0]: ix[0]+block_size]
train[ix[1]: ix[1]+block_size]

tensor([67,  1, 71, 53,  0, 52, 57, 55])

In [13]:
block_size = 8
batch_size = 4
def get_batched_data(split):
    data = train if 'train' else 'test'
    random_indices = torch.randint((len(data)-block_size), (batch_size,)) #random indices of batch size inside the data
    #print(random_indices)
    x = torch.stack([data[i:i+block_size]for i in random_indices]) #stacking a batch of 4 input block each of block size = 8 
    y = torch.stack([data[i+1:i+block_size+1]for i in random_indices]) #stacking a batch of 4 target block each of block size = 8 
    return x.to(device), y.to(device)

X,y = get_batched_data('train')
print(f'inputs: {X}, outputs: {y}')

inputs: tensor([[57, 62,  1, 23, 49, 62, 68, 53],
        [ 1, 61, 63, 66, 67, 53, 60,  1],
        [57, 53, 62, 52,  1, 68, 56, 53],
        [53, 64,  1, 55, 63, 63, 52,  1]], device='cuda:0'), outputs: tensor([[62,  1, 23, 49, 62, 68, 53,  5],
        [61, 63, 66, 67, 53, 60,  1, 63],
        [53, 62, 52,  1, 68, 56, 53,  1],
        [64,  1, 55, 63, 63, 52,  1, 54]], device='cuda:0')


In [16]:
class bigram_model(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #what is embedding matrix
        self.embedding_matrix = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets = None):
        #what is logits?
        logits = self.embedding_matrix(index)
        if targets is None:
            loss = None
        else:          
            
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self,index,max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]  #dim ain't dimensioning: it is now as targets are default to None so no loss in logits dim(3-dim)
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index

model = bigram_model(vocab_size = len(chars))
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


Kw;j)m5.wMNDi(§U7)JaTDê0rKiylTveYF"mPQ[jVæ*"ñG'ybZCn‡§£dCêyIjkO[m3A(ék'uFe,MLYB"a!&pECWO_]fpmhJtZP:rZ:Uw
‡æ"
.[£MmzrWr.bQK-pZ1RXIdVr.Z'RQfQKuæ"!wI.b7zgi.11.7bÑYBThcclgiqh3,.bpBæ[£?FGXM?QD.mmFhvF.1M:r-alATpHHFfrKu-zñU﻿Cæ[orZ﻿R£T†bY*ep£X‡æRoL]pXEXU&hZ0W.ikf†æn)S77fX ,R.†bÑéæL*&Z?vH§:_q?£3JYts£REVu-sHñ_qMKR,tshD5CnñS)(i-IwNMXêêYc_m-sLST?uoFf1bC()ê.q*)ñ0
lñ?f)M '75h'BsBVKnh﻿3Gou77RBB[Yö-†ylmPsIE
JFKa[ kfFf*Zpzñw gLZ5-;WK£:£†"æ.mmtslHiæ.J ÑNMBsGPBÑ1Rwse)lCX)EAJ byé_byé0§﻿Ppzgm†0gTvXpTMB?]1SbZ﻿﻿gSbmV]


### AdamW Optimizer

AdamW is a modification of Adam that adds weight decay regularization to the optimize
.
Weight decay helps prevent overfitting by adding a penalty for large weights.

In [14]:
@torch.no_grad()  #decorator for no grad since we are only caLculating the loss we do not need to compute gradient as no weight updation (optimization) happens here
def estimate_loss():
    out = {} 
    model.eval()  #eval model for model evaluation
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batched_data(split)
            logits, loss = model(X, Y)  #model pass without using any gradients
            losses[k] = loss.item()
        out[split] = losses.mean()  #averaging loss over iters
    model.train()  #puts model into training model: weights and biases are updated during this mode
    return out

In [18]:
##creating optimizer 
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250  #every 250 iterations evaluate the model

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        out = estimate_loss()
        print(f"Step: {iter}, Training loss: {out['train']:.4f}, Evaluation loss: {out['test']:.4f}")
    #sample from batch of data
    batched_x, batched_y = get_batched_data(split = 'train')
    #evaluating loss and performing back propagation
    logits, loss = model.forward(batched_x, batched_y)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()  #differentiate the loss fn wrt parameters
    optimizer.step() #update the parameters
    
print(loss.item()) #.item() get value from torch tensor

Step: 0, Training loss: 5.0383, Evaluation loss: 5.0281
Step: 250, Training loss: 4.9543, Evaluation loss: 4.9651
Step: 500, Training loss: 4.9154, Evaluation loss: 4.9131
Step: 750, Training loss: 4.8330, Evaluation loss: 4.8514
Step: 1000, Training loss: 4.7711, Evaluation loss: 4.7602
Step: 1250, Training loss: 4.7198, Evaluation loss: 4.7218
Step: 1500, Training loss: 4.6549, Evaluation loss: 4.6559
Step: 1750, Training loss: 4.5986, Evaluation loss: 4.5859
Step: 2000, Training loss: 4.5493, Evaluation loss: 4.5264
Step: 2250, Training loss: 4.4557, Evaluation loss: 4.4874
Step: 2500, Training loss: 4.4321, Evaluation loss: 4.4227
Step: 2750, Training loss: 4.3575, Evaluation loss: 4.3784
Step: 3000, Training loss: 4.2936, Evaluation loss: 4.3037
Step: 3250, Training loss: 4.2568, Evaluation loss: 4.2491
Step: 3500, Training loss: 4.1753, Evaluation loss: 4.2320
Step: 3750, Training loss: 4.1566, Evaluation loss: 4.1363
Step: 4000, Training loss: 4.1200, Evaluation loss: 4.1076
Ste

In [20]:
context = torch.zeros((1,1), dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


OayI‡"-†xQD﻿rpZ﻿bBonsla5£MQ?£†XochsPy﻿vHKHl†æXV;§ænxJF-d._q![heu£liswadysopFPu_0drenJXb
o:(ö,
Icöqir,ê-:§na?a?FeKokf.-0E£Is
l Ped gitiTz
oo b;L-
Tl0RX§Kæ]he e,X_Trk§﻿?G:HBg7kÑéw-;E:Yw(dIjênu﻿ lHitiSs;L-£5[_s
Yithg h?f_æyalBT n rerirkI]‡Iu﻿_7ê35VICnk-?d atl?v-"
O-Ñz!j-3riDX aicht&HcJ&cY''†Fd owl:êBld&Fl.7Pn‡11mma(DAIQ.q'-E,7( M5k.Na YbkeZBTlm5[?mE*exAPDCYswado ba
jH_KSY&PqejDecK
Ul yupts Bi£[!m‡P‡IX.ed m
h
Avu
ed slmty
7ÑzGo?Q?.bêTKnjUæPlÑYtog,OJ0.[?NPqI£s![Fh*A(æ:!æ]'t.71,§Xr
vHulse5Y§tykahonwar


### GPT Model

embedding = nn.Embedding(vocab_size, n_embedd)  #character level in context of the vocab size (wrt vocabulary)

positional_embedding = nn.Embedding(block_size, n_embedd) #postion of each character in a block input- position in context of each charater appearing in the block (wrt position of each character in a block)

## Module list and Sequential (difference): 
* Sequential processing: one block depends on the output of another block.
* Module list: runs each block isolated, computation processed in parallel, computations are structured to take advantage of GPU capabilities for simulataneous computations.

In [15]:
n_embedd  = 384 #number of total dims we want to capture from all heads concatenated together
n_decoders = 4  #4 decoder layers
dropout = 0.2 #20% neurons are dropped out to prevent overfitting
num_attention_heads= 4 # number of attention heads running in parallel
max_iters = 200  #number of training itterations
learning_rate = 1e-3 #for optimizer
eval_iters = 100 #for reporting loss
block_size = 8  #size of one input sequence
batch_size = 4  #how many lbock sized input sequences we want at the same time

In [16]:
#scaled dot product attention

#each head has 96 features and 4 heads in parallel = 384 features

class AttentionHead(nn.Module):

    def __init__(self, head_size):  # head_size-> n_embedd // num_attention_heads  384//4 -> 96
        super().__init__()
        self.key = nn.Linear(n_embedd, head_size, bias = False)  #384 -> 96 features
        self.query = nn.Linear(n_embedd, head_size, bias = False)
        self.value = nn.Linear(n_embedd, head_size, bias = False)
        #register buffer registers no look ahead masking in the model state
        #instaed of reinitializing it for every single head for every single forward and backward pass we add it to model state
        #saves computation
        #efficient way- reduces training time
        #training can be donw without this but it would take longer to complete
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))#(8,8)
        self.dropout = nn.Dropout(dropout) #20% neurons are dropped out

    def forward(self, x):
        #xshape: (B,T,C) --> (4,8,384)
        B,T,C = x.shape  #(4,8,384)
        k = self.key(x) # C or n_embedd (384) -> head_size(96)  (B,T,C) -> (B,T,hs)
        q = self.query(x) # (B,T,C= 384) -> (B,T,hs = 96)
        #calculating attention scores
        #key.transpose(-2,-1) - interchanging last two dims (B,hs,T) = (4, 96, 8)
        #SCALING: *key.shape[-1] = 96 -> sqrt(96) for 
        #query(B,T,hs) @ key(B,hs,T) -> (B, T, T)
        scores = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        #masked fill - repacing 0 to -inf for T timestamp
        #masked fill to prevent look ahead
        scores = scores.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        scores = F.softmax(scores, dim = -1) #(B,T,T) #confidence scores for prediction
        scores = self.dropout(scores)  
        v = self.value(x)  #(B,T,hs)
        out = scores @ v
        #print(f'shape after attention head: {out.shape}')
        return out

In [17]:
#keys and queries part will be implemented during dot product attention 

class MultiHeadAttention(nn.Module):
    def __init__(self, num_attention_heads, head_size): ##(4, 96)
        super().__init__()
        #bunch of heads in parallel for each head
        self.multiheads = nn.ModuleList([AttentionHead(head_size) for _ in range(num_attention_heads)]) #4 heads running in parallel
        #below transformation adds another set of learnable parameters (weights and bias)
        #kinda helps network learn more about the the text fed
        self.proj = nn.Linear(head_size * num_attention_heads, n_embedd)  #head_size(96) * num_attention_head(4) = 384 ---projected_to---> n_embedd(384)
        self.dropout = nn.Dropout(dropout)  #dropping out 20% of neurons

    def forward(self, x):
        out = torch.cat([head(x) for head in self.multiheads], dim = -1) #concatenation the out each head along the last dimension  (B,T,C)-> concatenate along channel dim
        #4 heads running in parallel each having output (B,T,hs) 
        # on concatenating across dim=-1 output shape = (B,T, 4*hs) -> (B,T,C) 
        out = self.dropout(self.proj(out))
        #print(f'out shape after multi head attention mech: {out.shape}')
        return out 

In [18]:
class FeedForward(nn.Module):
    def  __init__(self, n_embedd):
        super().__init__()
        self.netw = nn.Sequential(
            nn.Linear(n_embedd, 4 * n_embedd),
            nn.ReLU(),
            nn.Linear(4 * n_embedd, n_embedd),
            nn.Dropout(dropout),
        )
    def forward(self,x):
        return self.netw(x)

In [19]:
class Decoder_block(nn.Module):

    def __init__(self, n_embedd, num_attention_heads):  #input shape to decoder block -> (B, T, C)-(4,8,384)
        super().__init__()
        #n_head is the number of heads and embedd is the embedding dim
        #each head captures 96 features -> head size
        #4 attentions heads working in parallel
        #how many features are each of them capturing?
        head_size = n_embedd // num_attention_heads #is the number of features that each head will be capturing-> 384 // 4 - > 96 head_size
        self.sa = MultiHeadAttention(num_attention_heads, head_size)  #(4, 96)
        self.ffwd = FeedForward(n_embedd) #(384)
        self.ln1 = nn.LayerNorm(n_embedd) #(384) for add and norm
        self.ln2 = nn.LayerNorm(n_embedd) #(384) 

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)  #post norm arch -  Add and Norm. #skip connection 
        y = self.ffwd(x)
        x = self.ln2(x + y)  #residual
        #print(f'shape after Decode block: {x.shape}')
        return x

In [20]:
class GPTmodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #what is embedding matrix
        self.embedding_matrix = nn.Embedding(vocab_size, n_embedd)  #char level encoding art to vocab
        self.positional_matrix = nn.Embedding(block_size, n_embedd)  #char level positional embedding wrt to position of each char in a block
        
        self.decoder_blocks = nn.Sequential(*[Decoder_block(n_embedd, num_attention_heads  = num_attention_heads) for _ in range(n_decoders)])
        
        self.final_layer_norm = nn.LayerNorm(n_embedd)
        self.linear_head_final = nn.Linear(n_embedd, vocab_size)   #layer to get the next character given the input characters in hte vocab later softmax will be used to get the most probable character 
        
        #initializing weights with mean = 0.0 and  std dev = 0.02
        #helps training coverge better
        #weights are initialized properly
        self.apply(self._init_weights)

    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                  

    
    def forward(self, index, targets = None):
        #print(index.shape)
        B, T = index.shape

        
        token_embeddings = self.embedding_matrix(index)  ##index (4,8) (B,T) --token_embeddings--> (4,8,384)(B,T,C)
        pos_embeddings = self.positional_matrix(torch.arange(T, device = device))
        x = token_embeddings + pos_embeddings  #broadcasting helps- check broadcasting semantics in torch SHAPE - (4,8,384)
        #print(x.shape)
        x = self.decoder_blocks(x)
        x = self.final_layer_norm(x)
        #print(f'final output shape before linear layer: {x.shape}')
        logits = self.linear_head_final(x)
        #print(f'final output shape: {logits.shape}')

        
        #what is logits?
        #logits = self.embedding_matrix(index)
        if targets is None:
            print('FLAG')
            loss = None
        else:          
            
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            #print(f'shape after reshaping logits: {logits.shape} and targets shape: {targets.shape}')
            loss = F.cross_entropy(logits, targets)
            #print(loss.item())
            
        return logits, loss

    def generate(self,index,max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]  #dim ain't dimensioning: it is now as targets are default to None so no loss in logits dim(3-dim)
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index

model = GPTmodel(vocab_size = len(chars))
m = model.to(device)

# context = torch.zeros((1,1), dtype = torch.long, device= device)
# generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
# print(generated_chars)

In [21]:
# ##creating optimizer 
# max_iters = 1000
# learning_rate = 3e-4
# eval_iters = 250  #every 250 iterations evaluate the model

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        out = estimate_loss()
        print(f"Step: {iter}, Training loss: {out['train']:.4f}, Evaluation loss: {out['test']:.4f}")
    #sample from batch of data
    batched_x, batched_y = get_batched_data(split = 'train')
    #evaluating loss and performing back propagation
    logits, loss = m.forward(batched_x, batched_y)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()  #differentiate the loss fn wrt parameters
    optimizer.step() #update the parameters
    
print(loss.item()) #.item() get value from torch tensor

Step: 0, Training loss: 4.4786, Evaluation loss: 4.4998
Step: 100, Training loss: 2.9954, Evaluation loss: 3.0874
3.0235912799835205


In [ ]:
context = torch.zeros((1,1), dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)